In [2]:
import os
import sys
import traceback
from pathlib import Path
import tempfile
import uuid
from datetime import datetime
from flask import Flask, request, jsonify, render_template, send_from_directory

# Check Flask installation
try:
    from flask import Flask, request, jsonify, render_template, send_from_directory
except ImportError:
    print("Flask is not installed. Please install it with: pip install flask")
    sys.exit(1)

# Check moviepy installation
try:
    import moviepy
except ImportError:
    print("MoviePy is not installed. Please install it with: pip install moviepy")
    sys.exit(1)

# Check spacy installation
try:
    import spacy
    try:
        nlp = spacy.load("en_core_web_sm")
    except OSError:
        print("spaCy model 'en_core_web_sm' is not installed. Please install it with: python -m spacy download en_core_web_sm")
        sys.exit(1)
except ImportError:
    print("spaCy is not installed. Please install it with: pip install spacy")
    sys.exit(1)

# Initialize Flask app
app = Flask(__name__)

# Set up logging to console
import logging
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
app.logger.addHandler(handler)
app.logger.setLevel(logging.DEBUG)

# Update these paths to your video directories - use Path objects for cross-platform compatibility
ISL_VIDEOS_PATH = Path(r"N:\CAI\hackathon\Olabs_Final\Sign\words")
ALPHABET_VIDEOS_PATH = Path(r"N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips")

app.logger.info(f"ISL_VIDEOS_PATH: {ISL_VIDEOS_PATH}, exists: {ISL_VIDEOS_PATH.exists()}")
app.logger.info(f"ALPHABET_VIDEOS_PATH: {ALPHABET_VIDEOS_PATH}, exists: {ALPHABET_VIDEOS_PATH.exists()}")

# Create a temporary directory for output videos
TEMP_DIR = Path(tempfile.gettempdir()) / "isl_videos"
TEMP_DIR.mkdir(exist_ok=True)
app.logger.info(f"Temporary directory created at: {TEMP_DIR}")

def rule_based_srl(sentence):
    """
    Extract key words for ISL mapping using dependency parsing.
    Returns a list of lemmatized, lowercase words.
    """
    try:
        doc = nlp(sentence)
        words = []
        for token in doc:
            if token.is_alpha and token.dep_ not in ("det", "aux", "punct"):
                words.append(token.lemma_.lower())
        if not words:
            words = [token.lemma_.lower() for token in doc if token.is_alpha]
        return words
    except Exception as e:
        app.logger.error(f"Error in rule_based_srl: {e}")
        return [word.lower() for word in sentence.split() if word.isalpha()]

def get_isl_clip(word):
    """Get video clip for a word if it exists."""
    try:
        video_path = ISL_VIDEOS_PATH / f"{word}.mp4"
        if video_path.exists():
            return moviepy.VideoFileClip(str(video_path))
        else:
            app.logger.info(f"Missing ISL clip for: {word}")
            return None
    except Exception as e:
        app.logger.error(f"Error getting clip for word '{word}': {e}")
        return None

def get_alphabet_clip(letter):
    """Get video clip for an alphabet letter if it exists."""
    try:
        letter = letter.upper()
        video_path = ALPHABET_VIDEOS_PATH / f"{letter}.mp4"
        if video_path.exists():
            return moviepy.VideoFileClip(str(video_path))
        else:
            app.logger.info(f"Missing alphabet clip for: {letter}")
            return None
    except Exception as e:
        app.logger.error(f"Error getting clip for letter '{letter}': {e}")
        return None

def spell_word_with_alphabet(word):
    """Break a word into letters and get clips for each letter."""
    clips = []
    for letter in word.lower():
        if letter.isalpha():
            clip = get_alphabet_clip(letter)
            if clip:
                clips.append(clip)
    return clips

def find_isl_clips(words):
    """Find video clips for words or spell them with alphabet clips."""
    all_clips = []
    word_status = []
    
    for word in words:
        clip = get_isl_clip(word)
        if clip:
            all_clips.append(clip)
            word_status.append({"word": word, "found": True, "spelled": False})
        else:
            # If word not found, spell it using alphabet signs
            alphabet_clips = spell_word_with_alphabet(word)
            if alphabet_clips:
                all_clips.extend(alphabet_clips)
                word_status.append({"word": word, "found": False, "spelled": True})
            else:
                word_status.append({"word": word, "found": False, "spelled": False})
    
    return all_clips, word_status

def create_isl_video(sentence):
    """Convert sentence to ISL video."""
    try:
        isl_words = rule_based_srl(sentence)
        app.logger.info(f"Extracted words: {isl_words}")
        
        clips, word_status = find_isl_clips(isl_words)
        
        if not clips:
            app.logger.warning("No clips found for any words")
            return None, word_status
        
        # Create output video
        output_filename = f"isl_{uuid.uuid4().hex}.mp4"
        output_path = str(TEMP_DIR / output_filename)
        
        try:
            app.logger.info(f"Concatenating {len(clips)} clips")
            final_video = moviepy.concatenate_videoclips(clips, method="compose")
            app.logger.info(f"Writing video to {output_path}")
            final_video.write_videofile(output_path, codec="libx264", fps=24, logger=None)
            app.logger.info("Video created successfully")
            return output_path, word_status
        except Exception as e:
            app.logger.error(f"Error creating video: {str(e)}")
            app.logger.error(traceback.format_exc())
            return None, word_status
        finally:
            # Release resources
            for clip in clips:
                try:
                    clip.close()
                except:
                    pass
    except Exception as e:
        app.logger.error(f"Error in create_isl_video: {str(e)}")
        app.logger.error(traceback.format_exc())
        return None, []

@app.route('/')
def index():
    """Render the main HTML page"""
    return render_template("index.html")

@app.route('/convert', methods=['POST'])
def convert_text():
    """API endpoint to convert text to sign language video"""
    try:
        app.logger.info("Received conversion request")
        data = request.get_json()
        
        if not data or 'text' not in data:
            return jsonify({'error': 'No text provided'}), 400
        
        text = data['text'].strip()
        if not text:
            return jsonify({'error': 'Empty text provided'}), 400
        
        app.logger.info(f"Converting text: '{text}'")
        video_path, word_status = create_isl_video(text)
        
        if not video_path:
            return jsonify({
                'error': 'Could not create sign language video',
                'word_status': word_status
            }), 500
        
        # Generate URL for the video
        video_url = f"/video/{os.path.basename(video_path)}"
        
        return jsonify({
            'success': True,
            'video_url': video_url,
            'word_status': word_status
        })
    except Exception as e:
        app.logger.error(f"Error in convert_text: {str(e)}")
        app.logger.error(traceback.format_exc())
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/video/<filename>')
def serve_video(filename):
    """Serve the generated video file"""
    try:
        return send_from_directory(TEMP_DIR, filename)
    except Exception as e:
        app.logger.error(f"Error serving video {filename}: {str(e)}")
        return "Video not found", 404

if __name__ == "__main__":
    app.logger.info("Starting Flask application...")
    try:
        app.run(host='127.0.0.1', port=5002)
    except Exception as e:
        print(f"Error starting Flask app: {str(e)}")
        traceback.print_exc()


[2025-03-01 14:53:59,618] INFO in 1940395012: ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: False


ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: False
ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: False


[2025-03-01 14:53:59,618] INFO in 1940395012: ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: False


ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: False
ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: False


[2025-03-01 14:53:59,618] INFO in 1940395012: Temporary directory created at: C:\Users\MATHAN\AppData\Local\Temp\isl_videos


...

[2025-03-01 14:57:04,941] INFO in 1940395012: Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_3080a2ef5d554e739e1f1605917ddbb5.mp4


Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_3080a2ef5d554e739e1f1605917ddbb5.mp4
Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_3080a2ef5d554e739e1f1605917ddbb5.mp4


[2025-03-01 14:57:06,180] INFO in 1940395012: Video created successfully


Video created successfully
Video created successfully


127.0.0.1 - - [01/Mar/2025 14:57:06] "POST /convert HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2025 14:57:06] "GET /video/isl_3080a2ef5d554e739e1f1605917ddbb5.mp4 HTTP/1.1" 206 -


In [3]:
import subprocess

def extract_audio(video_path, audio_path='audio.wav'):
    """Extract audio from a video file and save it as a WAV file."""
    command = f"ffmpeg -i {video_path} -ab 160k -ac 2 -ar 44100 -vn {audio_path}"
    subprocess.run(command, shell=True, check=True)
    return audio_path

# Predefined paths
video_path = r"N:\CAI\hackathon\Olabs_Final\video.mp4"  # Change this to your desired input video path
output_audio = extract_audio(video_path)
print(f"Audio extracted successfully to: {output_audio}")

Audio extracted successfully to: audio.wav


In [ ]:
import os
import sys
import traceback
from pathlib import Path
import tempfile
import uuid
from datetime import datetime
import subprocess
from flask import Flask, request, jsonify, render_template, send_from_directory, send_file

# Check Flask installation
try:
    from flask import Flask, request, jsonify, render_template, send_from_directory, send_file
except ImportError:
    print("Flask is not installed. Please install it with: pip install flask")
    sys.exit(1)

# Check moviepy installation
try:
    import moviepy
    from moviepy import VideoFileClip
except ImportError:
    print("MoviePy is not installed. Please install it with: pip install moviepy")
    sys.exit(1)

# Check spacy installation
try:
    import spacy
    try:
        nlp = spacy.load("en_core_web_sm")
    except OSError:
        print("spaCy model 'en_core_web_sm' is not installed. Please install it with: python -m spacy download en_core_web_sm")
        sys.exit(1)
except ImportError:
    print("spaCy is not installed. Please install it with: pip install spacy")
    sys.exit(1)

# Initialize Flask app
app = Flask(__name__, static_folder='static')

# Set up logging to console
import logging
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
app.logger.addHandler(handler)
app.logger.setLevel(logging.DEBUG)

# Update these paths to your video directories - use Path objects for cross-platform compatibility
ISL_VIDEOS_PATH = Path(r"N:\CAI\hackathon\Olabs_Final\Sign\words")
ALPHABET_VIDEOS_PATH = Path(r"N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips")

app.logger.info(f"ISL_VIDEOS_PATH: {ISL_VIDEOS_PATH}, exists: {ISL_VIDEOS_PATH.exists()}")
app.logger.info(f"ALPHABET_VIDEOS_PATH: {ALPHABET_VIDEOS_PATH}, exists: {ALPHABET_VIDEOS_PATH.exists()}")

# Create temporary directories for output videos and uploaded files
TEMP_DIR = Path(tempfile.gettempdir()) / "isl_videos"
UPLOAD_DIR = Path(tempfile.gettempdir()) / "isl_uploads"
TEMP_DIR.mkdir(exist_ok=True)
UPLOAD_DIR.mkdir(exist_ok=True)
app.logger.info(f"Temporary directory created at: {TEMP_DIR}")
app.logger.info(f"Upload directory created at: {UPLOAD_DIR}")

def rule_based_srl(sentence):
    """
    Extract key words for ISL mapping using dependency parsing.
    Returns a list of lemmatized, lowercase words.
    """
    try:
        doc = nlp(sentence)
        words = []
        for token in doc:
            if token.is_alpha and token.dep_ not in ("det", "aux", "punct"):
                words.append(token.lemma_.lower())
        if not words:
            words = [token.lemma_.lower() for token in doc if token.is_alpha]
        return words
    except Exception as e:
        app.logger.error(f"Error in rule_based_srl: {e}")
        return [word.lower() for word in sentence.split() if word.isalpha()]

def get_isl_clip(word):
    """Get video clip for a word if it exists."""
    try:
        video_path = ISL_VIDEOS_PATH / f"{word}.mp4"
        if video_path.exists():
            return moviepy.VideoFileClip(str(video_path))
        else:
            app.logger.info(f"Missing ISL clip for: {word}")
            return None
    except Exception as e:
        app.logger.error(f"Error getting clip for word '{word}': {e}")
        return None

def get_alphabet_clip(letter):
    """Get video clip for an alphabet letter if it exists."""
    try:
        letter = letter.upper()
        video_path = ALPHABET_VIDEOS_PATH / f"{letter}.mp4"
        if video_path.exists():
            return moviepy.VideoFileClip(str(video_path))
        else:
            app.logger.info(f"Missing alphabet clip for: {letter}")
            return None
    except Exception as e:
        app.logger.error(f"Error getting clip for letter '{letter}': {e}")
        return None

def spell_word_with_alphabet(word):
    """Break a word into letters and get clips for each letter."""
    clips = []
    for letter in word.lower():
        if letter.isalpha():
            clip = get_alphabet_clip(letter)
            if clip:
                clips.append(clip)
    return clips

def find_isl_clips(words):
    """Find video clips for words or spell them with alphabet clips."""
    all_clips = []
    word_status = []
    
    for word in words:
        clip = get_isl_clip(word)
        if clip:
            all_clips.append(clip)
            word_status.append({"word": word, "found": True, "spelled": False})
        else:
            # If word not found, spell it using alphabet signs
            alphabet_clips = spell_word_with_alphabet(word)
            if alphabet_clips:
                all_clips.extend(alphabet_clips)
                word_status.append({"word": word, "found": False, "spelled": True})
            else:
                word_status.append({"word": word, "found": False, "spelled": False})
    
    return all_clips, word_status

def create_isl_video(sentence):
    """Convert sentence to ISL video."""
    try:
        isl_words = rule_based_srl(sentence)
        app.logger.info(f"Extracted words: {isl_words}")
        
        clips, word_status = find_isl_clips(isl_words)
        
        if not clips:
            app.logger.warning("No clips found for any words")
            return None, word_status
        
        # Create output video
        output_filename = f"isl_{uuid.uuid4().hex}.mp4"
        output_path = str(TEMP_DIR / output_filename)
        
        try:
            app.logger.info(f"Concatenating {len(clips)} clips")
            final_video = moviepy.concatenate_videoclips(clips, method="compose")
            app.logger.info(f"Writing video to {output_path}")
            final_video.write_videofile(output_path, codec="libx264", fps=24, logger=None)
            app.logger.info("Video created successfully")
            return output_path, word_status
        except Exception as e:
            app.logger.error(f"Error creating video: {str(e)}")
            app.logger.error(traceback.format_exc())
            return None, word_status
        finally:
            # Release resources
            for clip in clips:
                try:
                    clip.close()
                except:
                    pass
    except Exception as e:
        app.logger.error(f"Error in create_isl_video: {str(e)}")
        app.logger.error(traceback.format_exc())
        return None, []

def extract_audio(video_path):
    """Extract audio from a video file using moviepy."""
    try:
        app.logger.info(f"Extracting audio from video: {video_path}")
        output_filename = f"audio_{uuid.uuid4().hex}.wav"
        output_path = str(TEMP_DIR / output_filename)
        
        # Use moviepy to extract audio
        video = VideoFileClip(video_path)
        audio = video.audio
        if audio is None:
            app.logger.warning("No audio track found in video")
            return None
            
        audio.write_audiofile(output_path, codec='pcm_s16le', fps=44100, nbytes=2, logger=None)
        video.close()
        
        app.logger.info(f"Audio extracted to: {output_path}")
        return output_path
    except Exception as e:
        app.logger.error(f"Error extracting audio: {str(e)}")
        app.logger.error(traceback.format_exc())
        return None

@app.route('/')
def index():
    """Render the main HTML page"""
    return render_template("index.html")

@app.route('/convert', methods=['POST'])
def convert_text():
    """API endpoint to convert text to sign language video"""
    try:
        app.logger.info("Received conversion request")
        data = request.get_json()
        
        if not data or 'text' not in data:
            return jsonify({'error': 'No text provided'}), 400
        
        text = data['text'].strip()
        if not text:
            return jsonify({'error': 'Empty text provided'}), 400
        
        app.logger.info(f"Converting text: '{text}'")
        video_path, word_status = create_isl_video(text)
        
        if not video_path:
            return jsonify({
                'error': 'Could not create sign language video',
                'word_status': word_status
            }), 500
        
        # Generate URL for the video
        video_url = f"/video/{os.path.basename(video_path)}"
        
        return jsonify({
            'success': True,
            'video_url': video_url,
            'word_status': word_status
        })
    except Exception as e:
        app.logger.error(f"Error in convert_text: {str(e)}")
        app.logger.error(traceback.format_exc())
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/upload-video', methods=['POST'])
def upload_video():
    """Handle video upload and extract audio"""
    try:
        app.logger.info("Received video upload request")
        
        # Check if the request has a file part
        if 'video' not in request.files:
            return jsonify({'error': 'No video file in request'}), 400
            
        video_file = request.files['video']
        
        # Check if a file was actually selected
        if video_file.filename == '':
            return jsonify({'error': 'No video file selected'}), 400
            
        # Save the uploaded file temporarily
        filename = f"video_{uuid.uuid4().hex}{os.path.splitext(video_file.filename)[1]}"
        video_path = str(UPLOAD_DIR / filename)
        video_file.save(video_path)
        
        app.logger.info(f"Video saved to {video_path}")
        
        # Extract audio from the video
        audio_path = extract_audio(video_path)
        
        if not audio_path:
            return jsonify({'error': 'Failed to extract audio from video'}), 500
            
        # Generate URL for the audio file
        audio_url = f"/audio/{os.path.basename(audio_path)}"
        
        return jsonify({
            'success': True,
            'audio_url': audio_url,
            'message': 'Audio extracted successfully'
        })
    except Exception as e:
        app.logger.error(f"Error in upload_video: {str(e)}")
        app.logger.error(traceback.format_exc())
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/video/<filename>')
def serve_video(filename):
    """Serve the generated video file"""
    try:
        return send_from_directory(TEMP_DIR, filename)
    except Exception as e:
        app.logger.error(f"Error serving video {filename}: {str(e)}")
        return "Video not found", 404

@app.route('/audio/<filename>')
def serve_audio(filename):
    """Serve the extracted audio file"""
    try:
        return send_from_directory(TEMP_DIR, filename)
    except Exception as e:
        app.logger.error(f"Error serving audio {filename}: {str(e)}")
        return "Audio not found", 404

# Clean up temporary files periodically (consider adding a scheduled task)
def cleanup_temp_files():
    """Remove temporary files older than 1 hour"""
    try:
        current_time = datetime.now()
        for dir_path in [TEMP_DIR, UPLOAD_DIR]:
            for file_path in dir_path.glob('*'):
                file_age = current_time - datetime.fromtimestamp(file_path.stat().st_mtime)
                if file_age.total_seconds() > 3600:  # 1 hour
                    file_path.unlink()
                    app.logger.info(f"Removed old temporary file: {file_path}")
    except Exception as e:
        app.logger.error(f"Error cleaning up temporary files: {str(e)}")

if __name__ == "__main__":
    app.logger.info("Starting Flask application...")
    try:
        # Schedule cleanup task or run at startup
        cleanup_temp_files()
        app.run(host='127.0.0.1', port=5002)
    except Exception as e:
        print(f"Error starting Flask app: {str(e)}")
        traceback.print_exc()

[2025-03-01 16:19:45,873] INFO in 2111121185: ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: True


ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: True
ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: True
ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: True
ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: True


[2025-03-01 16:19:45,873] INFO in 2111121185: ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: True


ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: True
ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: True
ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: True
ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: True


[2025-03-01 16:19:45,873] INFO in 2111121185: Temporary directory created at: C:\Users\MATHAN\AppData\Local\Temp\isl_videos


...

Missing ISL clip for: homework
Missing ISL clip for: homework
Missing ISL clip for: homework
Missing ISL clip for: homework
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf58.76.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 454, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1.57, 'bitrate': 464, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_size': [480, 360], 'video_bitrate': 454, 'video_fps': 30.0, 'video_duration': 1.57, 'video_n_frames': 47}
c:\Python312\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-wi

[2025-03-01 16:20:36,900] INFO in 2111121185: Concatenating 74 clips


{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf58.76.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 683, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 0.6, 'bitrate': 699, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_size': [480, 360], 'video_bitrate': 683, 'video_fps': 30.0, 'video_duration': 0.6, 'video_n_frames': 18}
c:\Python312\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -i N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips\K.mp4 -loglevel error -f image2pipe -vf scale=480:

[2025-03-01 16:20:36,983] INFO in 2111121185: Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_af312953cd7644b8887097cbf7e79259.mp4


Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_af312953cd7644b8887097cbf7e79259.mp4
Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_af312953cd7644b8887097cbf7e79259.mp4
Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_af312953cd7644b8887097cbf7e79259.mp4
Writing video to C:\Users\MATHAN\AppData\Local\Temp\isl_videos\isl_af312953cd7644b8887097cbf7e79259.mp4
